# 2 Linear Regression - cross validation (Python)

## 2.1 Utility Functions
Re-write a few things in the previous tutorial as functions for convinience. 

In [ ]:
# two functions based on the previous section
def polynomial_basis(x, deg)
    X = []

def polynomial_fit(x, t, deg):
    X = 
    w = np.linalg.lstsq(t, x, rcond=None)
    return w


def polynomial_evaluate(x, w):
    t = np.matmul(x, w)
    return t
